In [227]:
import pandas as pd
import numpy as np
from scipy.stats import norm

In [228]:
state = "west"

#### Adjency Matrix

In [229]:
# import .csv file as dataframe
df = pd.read_csv('county_adjacency2010.csv')
df.drop(['countyname', 'neighborname'], axis=1, inplace=True)
unique_counties = df['fipscounty'].unique()

# create the adjectency matrix
adj_matrix = pd.DataFrame(0, index=unique_counties, columns=unique_counties)
for index, row in df.iterrows():
    if row['fipscounty'] != row['fipsneighbor']:
        adj_matrix.at[row['fipscounty'], row['fipsneighbor']] = 1
        adj_matrix.at[row['fipsneighbor'], row['fipscounty']] = 1
adj_matrix

,1001,1003,1005,1007,1009,1011,1013,1015,1017,1019,...,72141,72143,72145,72147,72149,72151,72153,78010,78020,78030
1001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1005,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1007,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1009,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72151,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
72153,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
78010,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
78020,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


### Mortality

In [230]:
import pyreadr

mortality = pyreadr.read_r(f'{state}/mortality.rdata')['mortality']
mortality = mortality[mortality['pop'] > 10]
mortality

,zip,mortality,pop
0,59001,0.298893,271.0
1,59002,0.416667,12.0
2,59003,0.257426,101.0
3,59006,0.306452,124.0
4,59007,0.347826,23.0
...,...,...,...
5957,99363,0.444444,27.0
5958,99371,0.296296,81.0
5959,99401,0.307692,39.0
5960,99402,0.291176,340.0


In [231]:
# import zip-code --> county
zc_county = pd.read_csv("zip_to_county2010.csv")
zc_county[zc_county.tot_ratio>0.5].zip.unique().size
y = pd.merge(mortality, zc_county, on='zip', how="left")
y = y[["county","mortality"]].dropna()#.set_index("county_x") #= mortality
y = y.groupby("county").mean().reset_index()
counties = list(y['county'].astype(int))
#counties.remove(51560)
y = np.array(y["mortality"])#[y["county"]!=51560])
W = np.array(adj_matrix[counties].loc[counties,:])


### Moran's I

In [232]:
def moran(y, w):
    n = len(y)
    y_bar = np.mean(y)
    S0 = np.sum((y - y_bar) ** 2)
    S1 = 0.0
    for i in range(n):
        for j in range(n):
            S1 += w[i, j] * (y[i] - y_bar) * (y[j] - y_bar)
    I = (n / S0) * (S1 / np.sum(w))
    EI = -1 / (n - 1)
    VI = (n * np.sum(w ** 2) - 2 * np.sum(w) + np.sum(w ** 2)) / (n * (n - 1) * S0 ** 2)
    z = (I - EI) / np.sqrt(VI)
    p = 2 * norm.sf(np.abs(z))
    return I, z, p

I, z, p = moran(y,W)
print(f"Census Region: {state}, Moran's I = {I} (p-value = {p})")

Census Region: west, Moran's I = 0.3176430033843211 (p-value = 0.9643271167068538)


In [233]:
z

0.04472423447255494